In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# EDA

import pandas as pd
import pandas_profiling

train = pd.read_csv('../input/titanic/train.csv')
# train.profile_report()

## 2.3 How to find the relevant variables

In [ ]:
# confirm the relationship of features and objective variable
import matplotlib.pyplot as plt
import seaborn as sns

plt.hist(train.loc[train['Survived'] == 0, 'Age'].dropna(), bins =30, alpha=0.5, label='0')
plt.hist(train.loc[train['Survived'] == 1, 'Age'].dropna(), bins =30, alpha=0.5, label='1')
plt.xlabel('Age')
plt.ylabel('count')
plt.legend(title='Survived')
plt.show()

In [ ]:
# SibSp and y
sns.countplot(x='SibSp', hue='Survived', data=train)
plt.legend(loc='upper right', title='Survived')
plt.show()

In [ ]:
# Parch and y
sns.countplot(x='Parch', hue='Survived', data=train)
plt.legend(loc='upper right', title='Survived')
plt.show()

In [ ]:
# Fare and y
plt.hist(train.loc[train['Survived']==0,'Fare'].dropna(), range=(0,250), bins=25, alpha=0.5, label='0')
plt.hist(train.loc[train['Survived']==1,'Fare'].dropna(), range=(0,250), bins=25, alpha=0.5, label='0')
plt.xlabel('Fare')
plt.ylabel('count')
plt.legend(title='Survived')
plt.xlim(-5,250)
plt.show()

In [ ]:
# Pclass and y
sns.countplot(x='Pclass', hue='Survived', data=train)
plt.show()

In [ ]:
# Sex and y
sns.countplot(x='Sex', hue='Survived', data=train)
plt.show()

In [ ]:
# Embarked and y
sns.countplot(x='Embarked', hue='Survived', data=train)
plt.show()

1. ## 2.4 Make new features (FamilySize, IsAlone)

In [ ]:
import numpy as np
import pandas as pd

# data preprocessing
train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')
gender_submission = pd.read_csv('../input/titanic/gender_submission.csv')
data = pd.concat([train, test], sort=False)
data['Sex'].replace(['male', 'female'], [0, 1], inplace=True)
data['Embarked'].fillna(('S'), inplace=True)
data['Embarked'] = data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)
data['Fare'].fillna(np.mean(data['Fare']), inplace=True)

age_avg = data['Age'].mean()
age_std = data['Age'].std()

np.random.randint(age_avg - age_std, age_avg + age_std)
data['Age'].fillna(data['Age'].median(), inplace=True) 
#print(data.head())
"""
delete_columns = ['Name', 'PassengerId', 'SibSp', 'Parch', 'Ticket', 'Cabin']
data.drop(delete_columns, axis=1, inplace=True)

train = data[:len(train)]
test = data[len(train):]
y_train = train['Survived']
X_train = train.drop('Survived', axis=1)
X_test = test.drop('Survived', axis=1)
X_train.head()
y_train.head()

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(penalty='l2', solver='sag', random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
"""
# new features FamilySize
import seaborn as sns
data['FamilySize']=data['Parch'] + data['SibSp'] +1
train['FamilySize'] = data['FamilySize'][:len(train)]
test['FamilySize'] = data['FamilySize'][len(train):]
sns.countplot(x='FamilySize', data=train, hue='Survived')


In [ ]:
# new feature IsAlone
data['IsAlone'] = 0
data.loc[data['FamilySize']==1, 'IsAlone']=1
train['IsAlone'] = data['IsAlone'][:len(train)]
test['IsAlone'] = data['IsAlone'][len(train):]

delete_columns = ['Name', 'PassengerId', 'SibSp', 'Parch', 'Ticket', 'Cabin']
data.drop(delete_columns, axis=1, inplace=True)
train = data[:len(train)]
test = data[len(train):]
y_train = train['Survived']
X_train = train.drop('Survived', axis=1)
X_test = test.drop('Survived', axis=1)
X_train.head()
# new feature + logistic regression
sub = pd.read_csv('../input/titanic/gender_submission.csv')

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(penalty='l2', solver='sag', random_state=0)
clf.fit(X_train, y_train)
y_pred_familysize_isalone = clf.predict(X_test)
sub['Survived'] = list(map(int, y_pred_familysize_isalone))
sub.to_csv('submission_familysize_isalone.csv', index=False)
sub.head()

"""
clf.fit(X_train.drop('FamilySize', axis=1), y_train)
y_pred_isalone = clf.predict(X_test.drop('FamilySize', axis=1))

sub['Survived'] = list(map(int, y_pred_isalone))
sub.to_csv('submission_isalone.csv', index=False)

sub.head()
"""


In [ ]:
## 2.5 Using LightGBM
import numpy as np
import pandas as pd

train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')
gender_submission = pd.read_csv('../input/titanic/gender_submission.csv')
data = pd.concat([train, test], sort=False)
data['Sex'].replace(['male', 'female'], [0, 1], inplace=True)
data['Embarked'].fillna(('S'), inplace=True)
data['Embarked'] = data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)
data['Fare'].fillna(np.mean(data['Fare']), inplace=True)
data['Age'].fillna(data['Age'].median(), inplace=True)
data['FamilySize'] = data['Parch'] + data['SibSp'] + 1
data['IsAlone'] = 0
data.loc[data['FamilySize'] == 1, 'IsAlone'] = 1

delete_columns = ['Name', 'PassengerId', 'Ticket', 'Cabin']
data.drop(delete_columns, axis=1, inplace=True)

train = data[:len(train)]
test = data[len(train):]

y_train = train['Survived']
X_train = train.drop('Survived', axis=1)
X_test = test.drop('Survived', axis=1)
# from sklearn.linear_model import LogisticRegression
# clf = LogisticRegression(penalty='l2', solver='sag', random_state=0)

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
# y_pred[:10]

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0, stratify=y_train)
categorical_features = ['Embarked', 'Pclass', 'Sex']

import lightgbm as lgb
lgb_train=lgb.Dataset(X_train, y_train, categorical_feature=categorical_features)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train, categorical_feature=categorical_features)

params = {'objective': 'binary'}

model = lgb.train(params, lgb_train, valid_sets=[lgb_train, lgb_eval], verbose_eval=10, num_boost_round=1000, early_stopping_rounds=10)
y_pred=model.predict(X_test, num_iteration=model.best_iteration)

In [ ]:
y_pred[:10]
y_pred = (y_pred>0.5).astype(int)
y_pred[:10]

In [ ]:
## 2.6 Change the hyper-parameters of LightGBM
params = {
    'objective':'binary',
    'max_bin': 300,
    'learning_rate': 0.05,
    'num_leaves':40   
}
lgb_train=lgb.Dataset(X_train, y_train, categorical_feature=categorical_features)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train, categorical_feature=categorical_features)
model = lgb.train(params, lgb_train, valid_sets=[lgb_train, lgb_eval], verbose_eval=10, num_boost_round=1000, early_stopping_rounds=10)
y_pred=model.predict(X_test, num_iteration=model.best_iteration)

In [ ]:
## Using Optuna to adjust hyper params
import optuna
from sklearn.metrics import log_loss
def objective(trial):
    params={
        'objective':'binary',
        'max_bin': trial.suggest_int('max_bin',255,500),
        'learning_rate':0.05,
        'num_leaves':trial.suggest_int('num_leaves',32,128)
    }
    lgb_train=lgb.Dataset(X_train, y_train, categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train, categorical_feature=categorical_features)
    model = lgb.train(params, lgb_train, valid_sets=[lgb_train, lgb_eval], verbose_eval=10, num_boost_round=1000, early_stopping_rounds=10)
    y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)
    score = log_loss(y_valid,y_pred_valid)
    return score

study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=40)

In [ ]:
study.best_params

In [ ]:
## In 2.6 is holdout validation
## In 2.7 is CV (cross validation)
train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')
gender_submission = pd.read_csv('../input/titanic/gender_submission.csv')
data = pd.concat([train, test], sort=False)
data['Sex'].replace(['male', 'female'], [0, 1], inplace=True)
data['Embarked'].fillna(('S'), inplace=True)
data['Embarked'] = data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype(int)
data['Fare'].fillna(np.mean(data['Fare']), inplace=True)
data['Age'].fillna(data['Age'].median(), inplace=True)
data['FamilySize'] = data['Parch'] + data['SibSp'] + 1
data['IsAlone'] = 0
data.loc[data['FamilySize'] == 1, 'IsAlone'] = 1

delete_columns = ['Name', 'PassengerId', 'Ticket', 'Cabin']
data.drop(delete_columns, axis=1, inplace=True)

train = data[:len(train)]
test = data[len(train):]

y_train = train['Survived']
X_train = train.drop('Survived', axis=1)
X_test = test.drop('Survived', axis=1)
## init ends
X_train.head()
from sklearn.model_selection import KFold
y_preds =[]
models=[]
oof_train = np.zeros((len(X_train),))
cv = KFold(n_splits=5, shuffle=True, random_state=0)

categorical_features=['Embarked', 'Pclass', 'Sex']
params = {
    'objective':'binary',
    'max_bin': 300,
    'learning_rate': 0.05,
    'num_leaves':40   
}
X_train.head()
for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    #print(train_index)
    X_tr = X_train.loc[train_index,:]
    X_val = X_train.loc[valid_index,:]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]
    
    lgb_train=lgb.Dataset(X_tr, y_tr, categorical_feature=categorical_features)
    lgb_eval=lgb.Dataset(X_val, y_val, reference=lgb_train, categorical_feature=categorical_features)
    model = lgb.train(params, lgb_train, valid_sets=[lgb_train, lgb_eval], verbose_eval=10, num_boost_round=1000, early_stopping_rounds=10)
    oof_train[valid_index]=model.predict(X_val, num_iteration=model.best_iteration)
    y_pred = model.predict(X_test, num_iteration=model.best_iteration)
    y_preds.append(y_pred)
    models.append(model)


In [ ]:
pd.DataFrame(oof_train).to_csv('oof_train_kfold.csv', index=False)
scores = [
    m.best_score['valid_1']['binary_logloss'] for m in models
]
score = sum(scores)/len(scores)
print(score)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred_oof = (oof_train>0.5).astype(int)
accuracy_score(y_train, y_pred_oof)

In [ ]:
len(y_preds) #5
y_preds[0][:10]
y_sub = sum(y_preds) / len(y_preds)
y_sub = (y_sub > 0.5).astype(int)
y_sub[:10]

In [ ]:
## How to partition the folds more reasonably?

from sklearn.model_selection import KFold
cv = KFold(n_splits=5, shuffle=True, random_state=0)
for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]

    print(f'fold_id: {fold_id}')
    print(f'y_tr y==1 rate: {sum(y_tr)/len(y_tr)}')
    print(f'y_val y==1 rate: {sum(y_val)/len(y_val)}')

In [ ]:
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train, y_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]

    print(f'fold_id: {fold_id}')
    print(f'y_tr y==1 rate: {sum(y_tr)/len(y_tr)}')
    print(f'y_val y==1 rate: {sum(y_val)/len(y_val)}')
    
# Then replace the cv

In [2]:
## 2.8 Using emsemble
!ls ../input/submit-files

submission_lightgbm_holdout.csv  submission_randomforest.csv
submission_lightgbm_skfold.csv


In [3]:
import pandas as pd
sub_lgbm_sk = pd.read_csv('../input/submit-files/submission_lightgbm_skfold.csv')
sub_lgbm_ho = pd.read_csv('../input/submit-files/submission_lightgbm_holdout.csv')
sub_rf = pd.read_csv('../input/submit-files/submission_randomforest.csv')

df = pd.DataFrame({'sub_lgbm_sk': sub_lgbm_sk['Survived'].values,
                  'sub_lgbm_ho': sub_lgbm_ho['Survived'].values,
                  'sub_rf': sub_rf['Survived'].values})
df.head()

,sub_lgbm_sk,sub_lgbm_ho,sub_rf
0,0,0,0
1,0,0,1
2,0,0,0
3,0,0,0
4,0,0,1


In [4]:
df.corr()

,sub_lgbm_sk,sub_lgbm_ho,sub_rf
sub_lgbm_sk,1.000000,0.883077,0.796033
sub_lgbm_ho,0.883077,1.000000,0.731329
sub_rf,0.796033,0.731329,1.000000


In [5]:
sub = pd.read_csv('../input/titanic/gender_submission.csv')
sub['Survived'] = sub_lgbm_sk['Survived'] + sub_lgbm_ho['Survived'] + sub_rf['Survived']
sub.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [6]:
sub['Survived'] = (sub['Survived']>=2).astype(int)
sub.to_csv('submission_lightgbm_ensemble.csv', index=False)
sub.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [ ]:
## THE END of Titanic
## Other Data Types
bureau.head()
## Table data
previous_loan_counts = bureau.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().rename(colums={'SK_ID_BUREAU':'previous_loan_counts'})
previous_loan_counts.head()

application_train = pd.merge(application_train, previous_loan_counts, on='SK_ID_CURR', how='left')
application_train['prevous_loan_counts'].fillna(0,inplace=True)
application_train.head()

In [8]:
## 3.3 With text data (3.1 table data, groupby, pd.merge (table join))
import pandas as pd
df=pd.DataFrame({'text':['I like kaggle very much', 'I do not like kaggle', 'I do really love machine learning']})
df

,text
0,I like kaggle very much
1,I do not like kaggle
2,I do really love machine learning


In [10]:
## Bag of words
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=u'(?u)\\b\\w+\\b')
bag=vectorizer.fit_transform(df['text'])
bag.toarray()

array([[0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1],
       [1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0],
       [1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0]])

In [12]:
print(vectorizer.vocabulary_)

{'i': 1, 'like': 4, 'kaggle': 2, 'very': 10, 'much': 7, 'do': 0, 'not': 8, 'really': 9, 'love': 5, 'machine': 6, 'learning': 3}


In [13]:
## TF-IDF (frequency)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

vectorizer = CountVectorizer(token_pattern=u'(?u)\\b\\w+\\b')
transformer = TfidfTransformer()

tf = vectorizer.fit_transform(df['text'])
tfidf = transformer.fit_transform(tf)
print(tfidf.toarray())

[[0.         0.31544415 0.40619178 0.         0.40619178 0.
  0.         0.53409337 0.         0.         0.53409337]
 [0.43306685 0.33631504 0.43306685 0.         0.43306685 0.
  0.         0.         0.56943086 0.         0.        ]
 [0.34261996 0.26607496 0.         0.45050407 0.         0.45050407
  0.45050407 0.         0.         0.45050407 0.        ]]


In [14]:
print(vectorizer.vocabulary_)

{'i': 1, 'like': 4, 'kaggle': 2, 'very': 10, 'much': 7, 'do': 0, 'not': 8, 'really': 9, 'love': 5, 'machine': 6, 'learning': 3}


In [15]:
## Word2Vec
from gensim.models import word2vec
sentences = [d.split() for d in df['text']]
model = word2vec.Word2Vec(sentences, size=10, min_count=1, window=2, seed=7)
model.wv['like']

array([-0.02067981,  0.0131102 ,  0.03317117,  0.00186954,  0.00616337,
        0.03904883, -0.01942172,  0.0012289 ,  0.03836093,  0.04205943],
      dtype=float32)

In [16]:
model.wv.most_similar('like')

[('do', 0.37898504734039307),
 ('really', 0.26278287172317505),
 ('love', 0.005604587495326996),
 ('machine', -0.10911547392606735),
 ('much', -0.15591022372245789),
 ('learning', -0.21832597255706787),
 ('very', -0.23330461978912354),
 ('not', -0.27250924706459045),
 ('I', -0.3530975580215454),
 ('kaggle', -0.37219277024269104)]

In [17]:
df['text'][0].split()

['I', 'like', 'kaggle', 'very', 'much']

In [18]:
import numpy as np
wordvec = np.array([model.wv[word] for word in df['text'][0].split()])
wordvec

array([[ 0.03017372, -0.02699223, -0.0012496 , -0.01339634, -0.00799477,
        -0.00531386, -0.04220407, -0.00395195, -0.02065495, -0.0157162 ],
       [-0.02067981,  0.0131102 ,  0.03317117,  0.00186954,  0.00616337,
         0.03904883, -0.01942172,  0.0012289 ,  0.03836093,  0.04205943],
       [ 0.01207318, -0.03283411, -0.01660173,  0.03145194,  0.01357688,
        -0.02567992, -0.0097396 ,  0.01921265, -0.0497833 ,  0.02829861],
       [-0.00382442, -0.00379456, -0.02855813, -0.04132524, -0.04324396,
        -0.04465462,  0.03896891, -0.01797059,  0.03008547,  0.01772794],
       [ 0.02605315, -0.02639901, -0.03170397,  0.02387284, -0.04093719,
         0.01489035,  0.02776833, -0.04471763, -0.0076779 ,  0.02981743]],
      dtype=float32)

In [19]:
np.mean(wordvec, axis=0)

array([ 0.00875916, -0.01538194, -0.00898845,  0.00049455, -0.01448713,
       -0.00434184, -0.00092563, -0.00923972, -0.00193395,  0.02043744],
      dtype=float32)

In [20]:
np.max(wordvec,axis=0)

array([0.03017372, 0.0131102 , 0.03317117, 0.03145194, 0.01357688,
       0.03904883, 0.03896891, 0.01921265, 0.03836093, 0.04205943],
      dtype=float32)